In [ ]:
# Black-Scholes

import numpy as np
from scipy.stats import binom
from scipy.stats import norm
import matplotlib.pyplot as plt
import time

# paramètres généraux
S0 = 140 # prix initial
T = 1.0 # temps final
K = 140 # strike
r = 0.1 # rendement
sigma = 0.1 # volatilité

In [ ]:
# Call Black-Scholes
def CallBS(S0,T,K,r,sigma):
    d1 = (np.log(S0/K)+(r+sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = (np.log(S0/K)+(r-sigma**2/2)*T)/(sigma*np.sqrt(T))
    C = S0*norm.cdf(d1)-K*np.exp(-r*T)*norm.cdf(d2)
    return C

In [ ]:
CallBS(S0,T,K,r,sigma)


In [ ]:
# Put Black-Scholes
def PutBS(S0,T,K,r,sigma):
    d1 = (np.log(S0/K)+(r+sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = (np.log(S0/K)+(r-sigma**2/2)*T)/(sigma*np.sqrt(T))
    P = K*np.exp(-r*T)*norm.cdf(-d2)-S0*norm.cdf(-d1)
    return P

In [ ]:
PutBS(S0,T,K,r,sigma)

In [ ]:
%timeit PutBS(S0,T,K,r,sigma) # estime le temps de calcul de la commande

In [ ]:
# autre solution - Put Black-Scholes
def PutBS2(S0,T,K,r,sigma):
    C = CallBS(S0,T,K,r,sigma)
    P = C-S0+K*np.exp(-r*T)
    return P

In [ ]:
PutBS2(S0,T,K,r,sigma)

In [ ]:
# Call CRR
def CallCRR(n,S0,T,K,r,sigma):
    up = np.exp(sigma*np.sqrt(T/n))
    R = np.exp(r*T/n)
    p = (R-(1/up))/(up-(1/up))
    ST = S0*up**np.arange(-n,n+1,2) # prix finaux de l'actif
    C = np.exp(-r*T)*np.dot(binom.pmf(range(n+1),n,p),np.maximum(ST-K,0)) # formule binomiale directe
    return C

In [ ]:
np.maximum(np.arange(-10,11,2),0)


In [ ]:
CallCRR(100,S0,T,K,r,sigma)

In [ ]:
%timeit CallCRR(1000000,S0,T,K,r,sigma)

In [ ]:
# autre solution - Call CRR
def CallCRR2(n,S0,T,K,r,sigma):
    up = np.exp(sigma*np.sqrt(T/n))
    R = np.exp(r*T/n)
    p = (R-(1/up))/(up-(1/up))
    q = p*up/R
    k = int(np.ceil((n+np.log(K/S0)/np.log(up))/2)) # indice immédiatement au-dessus du strike
    C = S0*(1-binom.cdf(k-1,n,q))-K*np.exp(-r*T)*(1-binom.cdf(k-1,n,p)) # somme binomiale partielle
    return C

In [ ]:
CallCRR2(100,S0,T,K,r,sigma)

In [ ]:
%timeit CallCRR2(1000000,S0,T,K,r,sigma) # beaucoup plus rapide, car n'assemble pas de vecteur de taille n

In [ ]:
# Put CRR
def PutCRR(n,S0,T,K,r,sigma):
    up = np.exp(sigma*np.sqrt(T/n))
    R = np.exp(r*T/n)
    p = (R-(1/up))/(up-(1/up))
    ST = S0*up**np.arange(-n,n+1,2) # prix finaux de l'actif
    P = np.exp(-r*T)*np.dot(binom.pmf(range(n+1),n,p),np.maximum(K-ST,0)) # formule binomiale directe
    return P

In [ ]:
PutCRR(100,S0,T,K,r,sigma)

In [ ]:
# autre solution - Put CRR
def PutCRR2(n,S0,T,K,r,sigma):
    up = np.exp(sigma*np.sqrt(T/n))
    R = np.exp(r*T/n)
    p = (R-(1/up))/(up-(1/up))
    q = p*up/R
    k = int(np.ceil((n+np.log(K/S0)/np.log(up))/2)) # indice immédiatement au-dessus du strike
    P = K*np.exp(-r*T)*(binom.cdf(k-1,n,p))-S0*(binom.cdf(k-1,n,q)) # somme binomiale partielle
    return P

In [ ]:
PutCRR2(100,S0,T,K,r,sigma)

In [ ]:
# autre autre solution - Put CRR
def PutCRR3(n,S0,T,K,r,sigma):
    C = CallCRR2(n,S0,T,K,r,sigma) # ou CallCRR, mais CallCRR2 est plus rapide
    P = C-S0+K*np.exp(-r*T)
    return P

In [ ]:
PutCRR3(100,S0,T,K,r,sigma)

In [ ]:
# convergence CRR => BS quand n -> +inf
BS = CallBS(S0,T,K,r,sigma) # valeur du call BS
N = 10000
CRR = np.zeros(N)
for n in range(N):
    CRR[n] = CallCRR2(n+1,S0,T,K,r,sigma) # valeur du call CRR pour n+1 étapes


In [ ]:
plt.plot(BS*np.ones(N),'b',CRR,'r')

In [ ]:
plt.loglog(abs(CRR[::2]-BS)) # convergence en O(1/n^2) car le droite est de pente -2

In [ ]:
plt.loglog(abs(CRR[1::2]-BS)) # convergence en O(1/n) car le droite est de pente -1

In [ ]:
plt.loglog(abs(CRR[::]-BS)) # convergence en O(1/n) dans le "pire des cas"
# pente -2 sur les indices pairs, -1 sur les indices impairs
# mais dans tous les cas (= pire des cas), on a une convergence en O(1/n)
# ce qui reste meilleur que la convergence théorique du
# théorème central limite en 1/sqrt(n) (qui donnerait une pente -1/2)